In [2]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

from src.three_way import evaluate_three_way

In [3]:
import numpy as np
import pandas as pd

from src.load_wdbc import load_uci_wdbc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X, y, ids = load_uci_wdbc("../data/raw/wdbc.data")
print("X shape:", X.shape)
print("Target distribution:\n", y.value_counts())

X shape: (569, 30)
Target distribution:
 1
1    357
0    212
Name: count, dtype: int64


In [4]:
alpha, beta = 0.91, 0.19

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rows = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    clf = LogisticRegression(max_iter=5000)
    clf.fit(X_train_s, y_train)

    y_pred = clf.predict(X_test_s)
    p_class1 = clf.predict_proba(X_test_s)[:, 1]  # P(benign)

    # Baseline metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    # 3WD decisions
    decision = np.where(
        p_class1 >= alpha, "Confirm_Benign",
        np.where(p_class1 <= beta, "Confirm_Malignant", "Uncertain")
    )

    non_uncertain_mask = decision != "Uncertain"
    n_total = len(y_test)
    n_non_uncertain = int(non_uncertain_mask.sum())
    coverage = n_non_uncertain / n_total
    uncertain_rate = 1 - coverage

    # Accuracy on non-uncertain subset
    if n_non_uncertain > 0:
        errors_non_uncertain = int((y_pred[non_uncertain_mask] != y_test.to_numpy()[non_uncertain_mask]).sum())
        acc_non_uncertain = 1 - (errors_non_uncertain / n_non_uncertain)
    else:
        errors_non_uncertain = 0
        acc_non_uncertain = np.nan

    rows.append({
        "fold": fold,
        "baseline_accuracy": acc,
        "baseline_precision": prec,
        "baseline_recall": rec,
        "baseline_f1": f1,
        "coverage": coverage,
        "uncertain_rate": uncertain_rate,
        "non_uncertain_accuracy": acc_non_uncertain,
        "non_uncertain_errors": errors_non_uncertain,
        "n_total": n_total,
        "n_non_uncertain": n_non_uncertain
    })

cv_df = pd.DataFrame(rows)
cv_df

,fold,baseline_accuracy,baseline_precision,baseline_recall,baseline_f1,coverage,uncertain_rate,non_uncertain_accuracy,non_uncertain_errors,n_total,n_non_uncertain
0,1,0.973684,0.985714,0.971831,0.978723,0.885965,0.114035,0.990099,1,114,101
1,2,0.947368,0.922078,1.000000,0.959459,0.894737,0.105263,1.000000,0,114,102
2,3,0.964912,0.947368,1.000000,0.972973,0.868421,0.131579,0.979798,2,114,99
3,4,0.991228,1.000000,0.986111,0.993007,0.903509,0.096491,1.000000,0,114,103
4,5,0.991150,0.986111,1.000000,0.993007,0.938053,0.061947,0.990566,1,113,106


In [5]:
summary = cv_df.drop(columns=["fold"]).agg(["mean", "std"]).T.reset_index()
summary.columns = ["metric", "mean", "std"]

display(summary)

cv_df.to_csv("../results/cv5_experiment1_folds.csv", index=False)
summary.to_csv("../results/cv5_experiment1_summary.csv", index=False)

print("Saved:")
print(" - results/cv5_experiment1_folds.csv")
print(" - results/cv5_experiment1_summary.csv")

,metric,mean,std
0,baseline_accuracy,0.973669,0.018590
1,baseline_precision,0.968254,0.032404
2,baseline_recall,0.991588,0.012576
3,baseline_f1,0.979434,0.014227
4,coverage,0.898137,0.025811
5,uncertain_rate,0.101863,0.025811
6,non_uncertain_accuracy,0.992093,0.008404
7,non_uncertain_errors,0.800000,0.836660
8,n_total,113.800000,0.447214
9,n_non_uncertain,102.200000,2.588436


Saved:
 - results/cv5_experiment1_folds.csv
 - results/cv5_experiment1_summary.csv
